In [1]:
import openai
import tensorflow as tf
import pandas as pd

In [4]:
with open('khan_labeled_data/train', 'r', encoding='utf-8') as f:
  data = f.read().splitlines()

In [5]:
labels = []
text = []
for line in data:
  labels.append(' ' + line[0])
  text.append(line[2:])

In [6]:
df = pd.DataFrame(zip(text, labels), columns = ['prompt','completion']) #[:300]
df.head()

,prompt,completion
0,"certainly , many algorithms are built complex ...",1
1,but what i would like to ask - what are the al...,1
2,is it just based on the length of code ?,0
3,is tere a way to solve a math problem with alg...,1
4,could dna be an algorithm ?,1


In [7]:
df.to_json("df.jsonl", orient='records', lines=True)

In [9]:
!openai tools fine_tunes.prepare_data -f df.jsonl -q

Analyzing...

- Your file contains 5236 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- All prompts end with suffix ` ?`

No remediations found.
- [Recommended] Would you like to split into training and validation set? [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified files to `df_prepared_train.jsonl` and `df_prepared_valid.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "df_prepared_train.jsonl" -v "df_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " 1"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` ?` for the

In [17]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [12]:
!openai api fine_tunes.create -t "df_prepared_train.jsonl" -v "df_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " 1" -m ada

Uploaded file from df_prepared_train.jsonl: file-4QOhi018KGejv6HNQAc0oFur
Uploaded file from df_prepared_valid.jsonl: file-ep0wHmbiirCeV4luYxzcVMVR
Created fine-tune: ft-HWkjULQb7UgpkNlaJ3fBy8sK
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-23 20:59:19] Created fine-tune: ft-HWkjULQb7UgpkNlaJ3fBy8sK

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-HWkjULQb7UgpkNlaJ3fBy8sK




Upload progress:   0%|          | 0.00/461k [00:00<?, ?it/s]
Upload progress: 100%|██████████| 461k/461k [00:00<00:00, 462Mit/s]

Upload progress:   0%|          | 0.00/110k [00:00<?, ?it/s]
Upload progress: 100%|██████████| 110k/110k [00:00<00:00, 109Mit/s]


In [15]:
!openai api fine_tunes.follow -i ft-HWkjULQb7UgpkNlaJ3fBy8sK

[2023-04-23 20:59:19] Created fine-tune: ft-HWkjULQb7UgpkNlaJ3fBy8sK

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-HWkjULQb7UgpkNlaJ3fBy8sK



In [20]:
openai.api_key = ''

In [21]:
ft_model = 'ada:ft-personal-2023-04-24-01-17-14'
sample_question = 'What is the very best day of the year?'
res = openai.Completion.create(model=ft_model, prompt=sample_question, max_tokens=1, temperature=0, logprobs=2)

In [23]:
print(res['choices'][0]['text'])
print(res)

 1
{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          38
        ],
        "token_logprobs": [
          -0.040069122
        ],
        "tokens": [
          " 1"
        ],
        "top_logprobs": [
          {
            " 0": -3.2578235,
            " 1": -0.040069122
          }
        ]
      },
      "text": " 1"
    }
  ],
  "created": 1682300098,
  "id": "cmpl-78fOU1A5woj3U2Sr6Xll5yn2PjufY",
  "model": "ada:ft-personal-2023-04-24-01-17-14",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 10,
    "total_tokens": 11
  }
}
